# Kahoot Ranking

Este Notebook filtra os vencedores do Kahoot, retornando os primeiros n colocados.

## ▶ Pontuação
### 1. Pontos de pódio
Para cada Kahoot:

🥇1 lugar : 3 pontos

🥈2 lugar : 2 pontos

🥉3 lugar : 1 ponto

### 2. Pontos do Kahoot
O desempate é feito pela pontuação acumulada dos Kahoots. 

In [28]:
import os
import pandas as pd
from unidecode import unidecode
from fuzzywuzzy import process
import re

import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def clean_string(input_string):
    # Use a regular expression to remove spaces and numbers
    cleaned_string = re.sub(r'[\s\d\W]', '', input_string)

    # Remove accents using unidecode
    string = unidecode(cleaned_string).lower()

    return string

def normalize_name(name, known_names, mapping, threshold=30):
    """
    Normalize a name by checking against a mapping and known names.

    Parameters:
    name (str): The name to normalize.
    known_names (list): A list of known names to compare against.
    mapping (dict): A dictionary mapping names to their normalized forms.
    threshold (int): The minimum score for a match to be considered valid (default is 30).

    Returns:
    str: The normalized name if a match is found; otherwise, returns 'johann'.
    """
    if name in mapping:
        return mapping[name]

    match = process.extractOne(name, known_names)

    if match and match[1] >= threshold:
        return match[0]

    return 'johann'

## Set folder path

In [29]:
# Set folder path
folder_path = "2024-4_O VERBO"

## Name exception dict 
Define name aliases to substitute (check which names have more than 1 alias)

'alias': 'originalname'

In [40]:
name_alias = {
    'mar+bi=marbi': 'mardabi',
    'marsembi': 'mardabi',
    'marbinoso': 'mardabi',
    'marcombi': 'mardabi',
    'bine': 'bidomar',
    'sabrine': 'bidomar',
    'mardomar': 'johann',
    'quadra': 'johann',
    'quadrado': 'johann',
    'bigxand': 'xandao',
    'shaquilleomeal': 'johann',
    'bobberkurwa': 'johann',
    'paracetamal': 'johann',
    'quadroh': 'johann',
    'vaixco ': 'johann',
    'bagriel': 'johann',
    'mardomar': 'johann',
    'quadrado': 'johann',
    'luaraa': 'luara',
    'luaraara': 'luara',
    'kakerlake': 'johann',
    'fmr': 'johann',
    'pirarucu': 'johann',
    'tucunare': 'johann',
    'bambu': 'johann',
    'yej!b': 'yejin',
    'yej!n': 'yejin',
    'tirisco': 'johann',
    'luu': 'luara',
    'luuu': 'luara',
    'lua': 'luara',
    'gih': 'giovanna',
}

## Main

In [41]:
# Main
# Get file info
file_list = [f for f in os.listdir(folder_path)]

df_dict = {}
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    key = file_name
    df_dict[key] = pd.read_excel(
        file_path, sheet_name='Final Scores', usecols="A:C", header=2
    )

# Create dataframe for podium
main_podium = pd.DataFrame()
# Join the files
for key in df_dict:
    podium = df_dict[key].rename(columns={df_dict[key].columns[0]: 'Podium'})
    main_podium = pd.concat([main_podium, podium])

# Rename columns
main_podium.rename(columns={'Total Score (points)': 'Points'}, inplace=True)
# Change data type
main_podium['Podium'] = main_podium['Podium'].astype(int)
main_podium['Points'] = main_podium['Points'].astype(int)

# Clean names
main_podium['Player'] = main_podium['Player'].apply(
    lambda x: clean_string(x))

# Substitute name aliases
main_podium['Player'] = main_podium['Player'].replace(name_alias)

# Assign Podium points
point_mapping = {1: 3, 2: 2, 3: 1}
main_podium['Podium_Points'] = main_podium['Podium'].map(point_mapping)

In [42]:
# [DEBUG] Display the unique players to verify
logging.debug(f'Unique Players raw:\n{sorted(main_podium['Player'].unique())}')

2025-02-19 07:59:10 [DEBUG] Unique Players raw:
['', 'allan', 'ange', 'anna', 'anne', 'arroz', 'barth', 'bidomar', 'brenda', 'bruna', 'delisa', 'delosangeles', 'eduardo', 'einstein', 'emmerick', 'evelyn', 'feriza', 'fernando', 'franca', 'frank', 'gabriel', 'gigio', 'gustavow', 'jd', 'jhes', 'johann', 'lambari', 'le', 'luara', 'mardabi', 'mari', 'matheus', 'meriley', 'natal', 'natalmatheus', 'nath', 'rafa', 'renato', 'roosewelt', 'salin', 'tobias', 'trentin', 'veruska', 'victor', 'xandao', 'yejn']


## Final ranking

In [43]:
# Create final ranking
rank = (main_podium.loc[:, ['Player', 'Podium_Points', 'Points']]
        .groupby(['Player'])
        .sum()
        .reset_index())
# Index starts at 1
rank.index = rank.index + 1

In [44]:
# [DEBUG] Display the unique players to verify
logging.debug(f'Unique Players:\n{sorted(main_podium['Player'].unique())}')

2025-02-19 07:59:13 [DEBUG] Unique Players:
['', 'allan', 'ange', 'anna', 'anne', 'arroz', 'barth', 'bidomar', 'brenda', 'bruna', 'delisa', 'delosangeles', 'eduardo', 'einstein', 'emmerick', 'evelyn', 'feriza', 'fernando', 'franca', 'frank', 'gabriel', 'gigio', 'gustavow', 'jd', 'jhes', 'johann', 'lambari', 'le', 'luara', 'mardabi', 'mari', 'matheus', 'meriley', 'natal', 'natalmatheus', 'nath', 'rafa', 'renato', 'roosewelt', 'salin', 'tobias', 'trentin', 'veruska', 'victor', 'xandao', 'yejn']


## Podium Variations

In [45]:
logging.info('Ranking por Podium Points e desempate por Points\n')

rank.sort_values(['Podium_Points', 'Points'],ascending=[False, False]) \
    .head(5) \
    .reset_index(drop=True)

2025-02-19 07:59:18 [INFO] Ranking por Podium Points e desempate por Points



,Player,Podium_Points,Points
0,brenda,12.0,42030
1,luara,8.0,22509
2,feriza,5.0,29496
3,ange,5.0,28979
4,natalmatheus,5.0,17576


In [46]:
# TOP 5 PODIUM PTS AND THEN BY ACC PTS
logging.info('Ranking por top 5 Points e ordenado por Podium Points')

rank.sort_values(['Points'], ascending=False) \
    .head(5) \
    .sort_values(['Podium_Points'], ascending=False) \
    .reset_index(drop=True)

2025-02-19 07:59:19 [INFO] Ranking por top 5 Points e ordenado por Podium Points


,Player,Podium_Points,Points
0,brenda,12.0,42030
1,feriza,5.0,29496
2,ange,5.0,28979
3,gabriel,4.0,31968
4,bruna,1.0,24355


In [47]:
logging.info('Ranking por Points')

rank.sort_values(['Points'], ascending=False) \
    .head(5) \
    .reset_index(drop=True)

2025-02-19 07:59:20 [INFO] Ranking por Points


,Player,Podium_Points,Points
0,brenda,12.0,42030
1,gabriel,4.0,31968
2,feriza,5.0,29496
3,ange,5.0,28979
4,bruna,1.0,24355


In [48]:
file_list

['O VERBO [01] Deus Encarnado.xlsx',
 'O VERBO [02] O início do ministério.xlsx',
 'O VERBO [03] Entrevista Secreta.xlsx',
 'O VERBO [04] A cidade de Sicar.xlsx',
 'O VERBO [05] Entrando em conflito.xlsx',
 'O VERBO [06] Curando um cego.xlsx',
 'O VERBO [07] O morto ressuscitado.xlsx',
 'O VERBO [08] À sombra da cruz.xlsx',
 'O VERBO [10] Discurso de despedida.xlsx',
 'O VERBO [11] A hora chegou.xlsx',
 'O VERBO [12] Condenação e crucifixão.xlsx']

In [50]:
# [DEBUG] Assiduidade | Presença
main_podium.groupby('Player')['Points'] \
           .count() \
           .sort_values(ascending=False) \
           .reset_index() \
           .rename(columns={'Points': 'Presença'})

,Player,Presença
0,allan,10
1,brenda,10
2,gabriel,9
3,bruna,8
4,feriza,8
5,ange,8
6,veruska,8
7,yejn,6
8,natal,5
9,mardabi,5
